# undistort_data_collection
カメラキャリブレーション用Notebookである`undistort/undistort.ipynb`、`undistort/undistort_stereo.ipynb`、`undistort/undistort_fisheye.ipynb`、`undistort/undistort_fisheye_stereo.ipynb`で使用する画像を撮影するNotebookです。

下記を参考に作成しています。  
https://github.com/NVIDIA-AI-IOT/jetbot/blob/master/notebooks/collision_avoidance/data_collection.ipynb

## 実行前の準備
キャリブレーションを始める前にチェスボードを準備します。下記リンクから画像をダウンロードして紙に印刷するか、スマホ等持ち運べるデバイスの画面に表示してください。紙に印刷した場合はチェスボードが曲がらないように丈夫な板や箱に貼り付けることをおすすめします。

https://github.com/opencv/opencv/blob/master/doc/pattern.png

`camera_param/camera_config.xml`を編集します。`<chess_size>20.5</chess_size>`の`20.5`の部分を使用しているチェスボードのマスのサイズに書き換えてください。

## キャリブレーション用画像の撮影
下記に続くセルを順に実行していきます。まずは必要なPythonモジュールを読み込み、Jetson Nano Mouseのカメラを起動します。

In [ ]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jnmouse import Camera, bgr8_to_jpeg
import xml.etree.ElementTree as ET
from uuid import uuid1

tree = ET.parse("../camera_param/camera_config.xml")
root = tree.getroot()
width = int(root.find("camera_param").find("width").text)
height = int(root.find("camera_param").find("height").text)

camera = Camera.instance(width=width, height=height, fps=5)
image = widgets.Image(format='jpeg', width=width, height=height)  # this width and height doesn't necessarily have to match the camera
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
image_display = widgets.Image(format='jpeg', width=width/2, height=height/2)
camera_link_display = traitlets.dlink((camera, 'value'), (image_display, 'value'), transform=bgr8_to_jpeg)

camera2 = Camera.instance(sensor_id=1, width=width, height=height, fps=5)
image2 = widgets.Image(format='jpeg', width=width, height=height)  # this width and height doesn't necessarily have to match the camera
camera_link2 = traitlets.dlink((camera2, 'value'), (image2, 'value'), transform=bgr8_to_jpeg)
image2_display = widgets.Image(format='jpeg', width=width/2, height=height/2)
camera_link_display = traitlets.dlink((camera2, 'value'), (image2_display, 'value'), transform=bgr8_to_jpeg)

実機を動かさなくてもNotebookの動作確認ができるように`chess_images_stereo`にサンプル画像を保存しました。ご自身のJetson Nano Mouseで撮影する際は下記セルを実行してサンプル画像を削除してください。

In [ ]:
!rm ./chess_images_stereo/*.jpg

すでにこのNotebookを一度実行済みで、`chess_images`ディレクトリにある画像を消したい場合は下記セルを実行してください。

In [ ]:
!rm ./chess_images/*.jpg

撮影画像保存用のディレクトリを作成します。左右片方ずつキャリブレーションを行う際は`images_dir`に文字列`chess_images`を代入します。ステレオカメラのキャリブレーションを行う際は`chess_images_stereo`を代入します。`images_dir`内の文字列は撮影画像の保存先にも用いるのでディレクトリが作成済みであってもこのセルは実行してください。

In [ ]:
import os

images_dir = 'chess_images'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(images_dir)

except FileExistsError:
    print('Directories not created becasue they already exist')

サンプル画像を用いて単眼カメラのキャリブレーションを行う場合は下記セルを実行してサンプル画像を`chess_images`ディレクトリにコピーしてください。

In [ ]:
!cp ./chess_images_stereo/*.jpg ./chess_images/

画像を撮影するボタンと撮影画像枚数を表示するテキストボックスを作成します。

In [ ]:
cap_button = widgets.Button(description='capture image')
cap_count = widgets.IntText(value=len(os.listdir(images_dir)))

撮影ボタンを押すと画像が保存されるように設定します。

In [ ]:
def save_snapshot():
    global images_dir, cap_count
    cap_count.value = len(os.listdir(images_dir))
    image_path_l = os.path.join(images_dir, str(uuid1()) + '_left.jpg')
    image_path_r = os.path.join(images_dir, str(uuid1()) + '_right.jpg')
    with open(image_path_l, 'wb') as fl, open(image_path_r, 'wb') as fr:
        fl.write(image.value)
        fr.write(image2.value)

cap_button.on_click(lambda x: save_snapshot())

下記セルを実行するとカメラ画像のプレビューと撮影ボタン、テキストボックスが表示されます。用意したチェスボードをカメラの前に構えて撮影してください。`capture image`ボタンをクリックすることで撮影できます。ボタン横のテキストボックスが撮影した画像の枚数です。テキストボックス内の数字が60くらいになるまで撮影することをお勧めします。

* 撮影の注意点
  * キャリブレーション用画像からチェスボードがはみ出ないように気を付けてください。市松模様の周囲にある余白も含めて撮影します。
  * ステレオカメラのキャリブレーションを行う際は両方のカメラの画角にチェスボードが収まるように撮影してください。
  * 光の反射によってチェスボードが白飛びしないようにしてください。
  * 撮影するチェスボードは様々な位置姿勢であることが好ましいです。同じ位置姿勢で撮影せずに遠ざけたり近づけたり、傾けたりしながら撮影するとキャリブレーションの品質が良くなります。

In [ ]:
display(widgets.HBox([image_display, cap_count, cap_button]))
display(image2_display)